<a href="https://colab.research.google.com/github/davidashirov/uzum_biz_an/blob/main/homeworks/07/mnist_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя работа 7: Знакомство с набором данных MNIST. Построение модели KNeighborsClassifier

### Основы API

Чаще всего использование API Scikit-Learn Estimator происходит следующим образом:

1. Выбирается класс модели, импортировав соответствующий класс из Scikit-Learn.
2. Подбираем значения гиперпараметров модели, инстанцировав класс модели с нужными значениями параметров.
3. Разделение данные на матрицу признаков и целевой вектор.
4. Настройка модели посредством метода `fit` экземпляра модели.
5. Применение модели к новым данным:
   - Для `supervised learning` (обучение с учителем) мы предсказываем метки для неизвестных данных с помощью метода `predict`.
   - Для `unsupervised learning` (обучение без учителя) мы преобразуем или оцениваем свойства примеров из набора данных с помощью методов `transform` или `predict`.

Согласно вышеуказнной последовательность требуется выполнить следующее задание:

1. Загрузить набор данных [MNIST]( http://yann.lecun.com/exdb/mnist/ )
2. Исследовать его. Вывести разнообразные числовые характериcтики набора данных, а также визуализировать отдельные элементы данных.
3. Разработать программный код, который позволит построить модель KNeighborsClassifier
4. Вывести метрики Confusion Matrix, Precision, Recall
5. Описать полученные результаты

Бонус
    На втором семинаре было дано введение в объектно-ориентированное программирование, попробуйте реализовать собственный класс модели.
    Реализация должна поддерживать API Scikit-Learn, что позволит использовать возможности Scikit-Learn


In [7]:
import os

files = ['train-images-idx3-ubyte',
         'train-labels-idx1-ubyte',
         't10k-images-idx3-ubyte'
         't10k-labels-idx1-ubyte']
os.system('mkdir input')
os.system('cd input')
for fname in files:
  os.system('wget -N http://yann.lecun.com/exdb/mnist/{fname}.gz -P /input/')
  os.system('!unzip train-images-idx3-ubyte.gz')

0

0

In [1]:
import numpy as np # linear algebra
import struct
from array import array
from os.path  import join

#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath

    def read_images_labels(self, images_filepath, labels_filepath):
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())

        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img

        return images, labels

    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)

In [2]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

#
# Set file paths based on added MNIST Datasets
#
input_path = 'input'
training_images_filepath = join(input_path, 'train-images-idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte')
test_images_filepath     = join(input_path, 't10k-images-idx3-ubyte')
test_labels_filepath     = join(input_path, 't10k-labels-idx1-ubyte')

mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()


FileNotFoundError: ignored